# Boxplot on AssessmentScore data, overlay w dots
In this notebook, I experiment my way into

1. [Set up](https://www.codingforentrepreneurs.com/blog/use-django-in-jupyter)
   Jupyter for Django:
   - I copied in the file `django_for_jupyter.py` in project root directory.
   - Same location, now issue `DJANGO_PROJECT="box_whiskers_demo" jupyter lab`.
   - This will start Jupyter lab **in browser** - and thus 
     access to **this Notebook** with Django project access.
     Does logic seem backward? Sorry!
1. **Retrieve data** from model `AssessmentScore`:
1. Preprocess (if necessary) in order to **prepare for 12-box-plot**.
1. Plot and label
1. **Overlay** with arbitraty, individual student data (**dot** plot).

With these preparations in place, we can **initiate** the **Django environment**:

In [1]:
from django_for_jupyter import init_django
init_django()

Now, the Notebook simulates the environment that exists inside the Django app.

We'll check out how many students are active and registered in the `1test` class.
Then, we'll inspect the names of the first two (counted in the order they are registered to the Django model).

In [2]:
from boxplot.models import Elev, Klasse
query_set = Elev.objects.filter( klasse = Klasse.objects.get( navn = '1test' ))
klasse_comprehension = [e.fulde_navn for e in query_set]
len(klasse_comprehension), klasse_comprehension[:2]

(23, ['Camilla Horserød', 'Camilla Vejlebyskov'])

The `filter()` returns a QuerySet containing no, one or multiple records,
while the `get()` method is assured to return **at most one** record in the QuerySet
or throw an error.

Thus, we may have **several students**, `Elev`,
but they all belong to the same, i.e. **exactly one, class**.
But we may only have students, provided a class with that name exists, **and** 
any students are registrered with that class.
Otherwise, we will have no students, but furthermore, 
the model manager will 
[raise](https://realpython.com/python-exceptions/)
the `DoesNotExist` error.

In [11]:
klassens_betegnelse = 'non_existant'
from django.core.exceptions import ObjectDoesNotExist
try: 
    query_set = Elev.objects.filter( klasse = Klasse.objects.get( navn = klassens_betegnelse ))
except ObjectDoesNotExist as e:
    print('Ups - "'+klassens_betegnelse+'" - findes ikke i Django-databasen.')

Ups - "non_existant" - findes ikke i Django-databasen.


Every **Django Model** has a `DoesNotExist` exception,
subclassing the `ObjectDoesNotExist` class, see 
[this SO answer](https://stackoverflow.com/a/52455861/888033).
Thus, the above construction **will catch** a `DoesNotExist` exception
of an **existing** class being presented to a `Elev` model where that class is registered nowhere.
It will **also catch** the demonstrated exception, where to class with that name is found.

Now, we shall select an assignment, `Aflevering`, followed by picking the `AssessmentScores` related to the QuerySet of `Elev` objects and to that `Aflevering`.

In [15]:
from boxplot.models import Elev, Klasse, AssesmentScore, Aflevering
[a.titel for a in Aflevering.objects.all()]

['Model af CoVID19']

OK, we have exactly 1 assignment, the `Aflevering` QuerySet has 1 element.
I think we can speak of the Django `AssementScore` model as a 
[reification](https://www.techopedia.com/definition/21674/reification)
of the relation between the Django `Aflevering` model and the `Elev` model.

![Reification UML](UML_assessmentScore_reification.png)

In [19]:
roster = Elev.objects.filter( klasse = Klasse.objects.get(navn='1test'))
len(roster), type([e for e in roster][4])

(23, boxplot.models.Elev)

In [23]:
submission = Aflevering.objects.filter( klasse = Klasse.objects.get(navn='1test'), titel__icontains='covid')

When filter should match *any* of the items in a list (thanks to
[this SO answer](https://stackoverflow.com/a/34303446/888033)):

In [28]:
scores = AssesmentScore.objects.filter(elev__in=roster)
len(scores), type([s for s in scores][2])

(23, boxplot.models.AssesmentScore)

In [27]:
import pandas as pd
df = DataFrame()

[<AssesmentScore: Opgave 'Model af CoVID19' fra Camilla Horserød: [13, 12, 9, 16, 15, 9, 14, 14, 14, 11, 7, 14]>,
 <AssesmentScore: Opgave 'Model af CoVID19' fra Camilla Vejlebyskov: [15, 13, 14, 15, 15, 19, 15, 17, 17, 15, 10, 17]>,
 <AssesmentScore: Opgave 'Model af CoVID19' fra Emma Oue: [21, 20, 13, 6, 23, 14, 15, 16, 18, 16, 16, 16]>,
 <AssesmentScore: Opgave 'Model af CoVID19' fra Helle Barup: [16, 14, 10, 13, 14, 11, 13, 13, 16, 16, 8, 12]>,
 <AssesmentScore: Opgave 'Model af CoVID19' fra Helle Byskov: [20, 19, 16, 18, 17, 12, 15, 17, 19, 18, 15, 18]>,
 <AssesmentScore: Opgave 'Model af CoVID19' fra Henrik Menstrup: [18, 6, 6, 11, 10, 6, 9, 9, 11, 11, 7, 10]>,
 <AssesmentScore: Opgave 'Model af CoVID19' fra Henrik Voer Hede: [19, 17, 16, 21, 17, 20, 17, 18, 21, 18, 14, 22]>,
 <AssesmentScore: Opgave 'Model af CoVID19' fra Ida Himmark Mark: [24, 11, 16, 19, 10, 8, 12, 12, 20, 16, 14, 14]>,
 <AssesmentScore: Opgave 'Model af CoVID19' fra Jens Febbersted: [18, 17, 16, 22, 20, 9, 15

Now I have to get the **last** six (hence the `-`) field names of the `AssessmentScore` model, stolen from
[this SO answer](https://stackoverflow.com/a/11075898/888033)

In [41]:
[field.name for field in AssesmentScore._meta.fields][-6:]

['tankegang', 'fagsprog', 'cas', 'diagram', 'sammenhæng', 'konklusion']

In [42]:
col_name_list = [f'opg{i}' for i in range(1,7)]
col_name_list += [field.name for field in AssesmentScore._meta.fields][-6:]
col_name_list

['opg1',
 'opg2',
 'opg3',
 'opg4',
 'opg5',
 'opg6',
 'tankegang',
 'fagsprog',
 'cas',
 'diagram',
 'sammenhæng',
 'konklusion']

In [63]:
import pandas as pd
pd.DataFrame(data=scores.values(), columns=col_name_list, index=[s.elev.fulde_navn for s in scores])

,opg1,opg2,opg3,opg4,opg5,opg6,tankegang,fagsprog,cas,diagram,sammenhæng,konklusion
Camilla Horserød,13,12,9,16,15,9,14,14,14,11,7,14
Camilla Vejlebyskov,15,13,14,15,15,19,15,17,17,15,10,17
Emma Oue,21,20,13,6,23,14,15,16,18,16,16,16
Helle Barup,16,14,10,13,14,11,13,13,16,16,8,12
Helle Byskov,20,19,16,18,17,12,15,17,19,18,15,18
Henrik Menstrup,18,6,6,11,10,6,9,9,11,11,7,10
Henrik Voer Hede,19,17,16,21,17,20,17,18,21,18,14,22
Ida Himmark Mark,24,11,16,19,10,8,12,12,20,16,14,14
Jens Febbersted,18,17,16,22,20,9,15,17,19,17,14,20
Jens Molshuse,19,18,15,22,23,18,20,18,23,20,15,19


In [62]:
[s.elev.fulde_navn for s in scores]

['Camilla Horserød',
 'Camilla Vejlebyskov',
 'Emma Oue',
 'Helle Barup',
 'Helle Byskov',
 'Henrik Menstrup',
 'Henrik Voer Hede',
 'Ida Himmark Mark',
 'Jens Febbersted',
 'Jens Molshuse',
 'Jørgen Hallenslev Gårde',
 'Jørgen Normark',
 'Jørgen Storå',
 'Jørgen Trængstrup',
 'Kirsten Dybdal',
 'Kirsten Hjortegårde',
 'Maria Skibsted Å',
 'Mette Blushøj',
 'Mette Tise Udflyttere',
 'Peter Holmdrup',
 'Peter Villendrup',
 'William Magleholm',
 'William Trævel Å']